In [11]:
from modeller import *
from modeller.automodel import *
from Bio import SeqIO
from Bio.SeqUtils import seq1
import os

#from modeller import soap_protein_od


pdb_code = '4wkq'
ligand_name_list = ['IRE-A','AAA-A']
output_path = "/Users/rcheeter/Desktop/MODELLER-OUTPUT-GLY"
pdb_file_path = "/users/rcheeter/Desktop/4wkq.pdb"


# FIX GET OF PDB CODE -- MUST EITHER BE IN HEADER LINE OR FIND ANOTHER WAY TO GET PDB CODE
# CANNOT RELY ON HEADER LINE
# ALSO FIX IN PGN PROGRAM (GET PDB AND GET LIGPLOT ESPECIALLY)

def get_alignment_pir(pdb_code,pdb_file_path,ligand_name_list,output_path):
    print(f'\n[running get_alignment_pir for [{pdb_file_path}]]')

    # import PDB file
    try:
        with open(pdb_file_path,'r') as pdb_file:
            pdb = pdb_file.readlines()
        n = len(pdb)
        if n==0:
            raise Exception(f'● ERROR: error opening PDB file; invalid PDB file path')
    except:
        raise Exception(f'● ERROR: error opening PDB file: [{pdb_file_path}]; invalid PDB file path') 
    
    if type(output_path)!=str or ' ' in output_path:
        raise Exception(f'● ERROR: error reading output path: [{output_path}]; must be a string without spaces')
    try:
        if not os.path.exists(output_path):
            os.makedirs(output_path)
            print(f'● NOTE: creating output path: [{output_path}]')
    except:
        raise Exception(f'● ERROR: error reading output path: [{output_path}]')

    # save original PDB file
    original_pdb_output_path = f'{output_path}/{pdb_code}_original'
    try:
        if not os.path.exists(original_pdb_output_path):
            os.makedirs(original_pdb_output_path)
            print(f'● NOTE: creating original PDB output path: [{original_pdb_output_path}]')
    except:
        raise Exception(f'● ERROR: error creating original PDB output path: [{original_pdb_output_path}]')
    
    original_pdb_file_path = f'{original_pdb_output_path}/{pdb_code}.pdb'
    open(original_pdb_file_path,'w').write(''.join(pdb))
    print(f'● NOTE: saving original PDB file to original PDB output path: [{original_pdb_file_path}]')
    
    # modify PDB file
    residue_name_list = ['ALA','VAL','ILE','LEU','MET','PHE','TYR','TRP','CYS','GLY',
                         'PRO','SER','THR','ASN','GLN','ARG','HIS','LYS','ASP','GLU']
    ligand_name_pdb_set = []
    
    n = len(pdb)
    i = 0
    while i<n:
        line = pdb[i]
        if line[0:6].upper()=='HETATM':
            ligand_name = f"{line[17:20].replace(' ','').upper()}-{line[21].upper()}"
            ligand_name_pdb_set.append(ligand_name)
            
        if (line[0:6].upper() in ['ATOM','ANISOU']) and (line[17:20].replace(' ' ,'').upper() not in residue_name_list):
            atom_name = line[12:16].replace(' ','').upper()
            residue_number = int(line[22:26])
            residue_name = line[17:20].replace(' ','').upper()
            chain_id = line[21].upper()
            
            # residue_name_1L = seq1(residue_name_3L)
            # removed_residues.append([residue_number,residue_name_3L,residue_name_1L])
            
            print(f'● WARNING: unrecognized ATOM/ANISOU residue: atom [{atom_name}] in residue [{residue_name}-{chain_id} #{residue_number}]; removing atom')
            pdb.remove(line)
            i -= 1
            n -= 1
            
        elif (line[0:6].upper()=='HETATM') and ((f"{line[17:20].replace(' ','').upper()}-{line[21].upper()}") not in ligand_name_list):
            atom_name = line[12:16].replace(' ','').upper()
            residue_number = int(line[22:26])
            residue_name = line[17:20].replace(' ','').upper()
            chain_id = line[21].upper()
            
            # residue_name_1L = seq1(residue_name_3L)
            # removed_residues.append([residue_number,residue_name_3L,residue_name_1L])

            print(f'● WARNING: unrecognized HETATM residue: atom [{atom_name}] in residue [{residue_name}-{chain_id} #{residue_number}]; removing atom')
            pdb.remove(line)
            i -= 1
            n -= 1
            
        i += 1
    
    # removed_residues = [list(residue) for residue in set(tuple(residue) for residue in removed_residues)]
    # print(removed_residues)
    
    ligand_name_pdb_set = list(set(ligand_name_pdb_set))
    ligand_name_list = list(set(ligand_name_list))
    ligand_name_difference = list(set(ligand_name_list)-set(ligand_name_pdb_set))
    
    if len(ligand_name_difference)!=0:
        ligand_name_difference = ', '.join(ligand_name_difference)
        print(f'● WARNING: ligand name(s) not found in PDB: [{ligand_name_difference}]; removing from ligand name list')
    
    ligand_name_list = [ligand_name for ligand_name in ligand_name_list if ligand_name not in ligand_name_difference]

    # save modified PDB file
    modified_pdb_output_path = f'{output_path}/{pdb_code}_modified'
    try:
        if not os.path.exists(modified_pdb_output_path):
            os.makedirs(modified_pdb_output_path)
            print(f'● NOTE: creating modified PDB output path: [{modified_pdb_output_path}]')
    except:
        raise Exception(f'● ERROR: error creating modified PDB output path: [{modified_pdb_output_path}]')
        
    modified_pdb_file_path = f'{modified_pdb_output_path}/{pdb_code}_mod.pdb'
    open(modified_pdb_file_path,'w').write(''.join(pdb))
    print(f'● NOTE: saving modified PDB file to modified PDB output path: [{modified_pdb_file_path}]')

    # create template and target sequences from PDB file
    template_sequence = ''
    with open(modified_pdb_file_path,'r') as modified_pdb_file:
        for record in SeqIO.parse(modified_pdb_file,'pdb-atom'):   
            template_sequence += ('/' + str(record.seq).replace('X','/').replace(' ',''))
        if pdb_code!=(record.id.split(':')[0].lower()):
            raise Exception('● ERROR: error parsing modified PDB file; PDB code does not mach original PDB code')
    
    template_sequence = list(template_sequence)[1:]
    for i in range(1,len(template_sequence)):
        if template_sequence[i]=='/' and (template_sequence[i-1] in ['/','-']):
            template_sequence[i] = '-'
    template_sequence = ''.join(template_sequence)
    
    for ligand_name in ligand_name_list:
        template_sequence += '/.'
    template_sequence += '*'
    
    n = 75
    template_sequence = [template_sequence[i:i+n] for i in range(0,len(template_sequence),n)]
    template_name = f'{pdb_code}_template'
    template_info = f'structure:{pdb_code}_mod.pdb   :FIRST:@:  END: :::     :     '

    template_text = [f'>P1;{template_name}',template_info]
    template_text.extend(template_sequence)
    template_text = '\n'.join(template_text)
    
    target_sequence = template_sequence
    target_name = f'{pdb_code}_target'
    target_info = f'sequence :{target_name}:     : :     : :::     :     '
    
    target_text = [f'>P1;{target_name}',target_info]
    target_text.extend(target_sequence)
    target_text = '\n'.join(target_text)
    
    pir_text = template_text + '\n\n' + target_text + '\n'
    
    
    pir_file_path = f'{output_path}/alignment.ali'
    open(pir_file_path,'w').write(pir_text)
    print(f'● NOTE: saving PIR sequence alignment to output path: [{pir_file_path}]')

    return pir_file_path,modified_pdb_file_path,target_name,template_name
    
    
    # mutation = [startind,endind,newseq]
# def make_mutation(pir_file_path,mutation,target_name):
    
#     return pir_file_path


def get_modeller(pir_file_path,target_name,template_name,output_path):

    os.chdir(output_path)
    env = Environ()
    env.io.atom_files_directory = [f'{output_path}/4wkq_modified']
    env.io.hetatm = True

    a = AutoModel(env, alnfile=pir_file_path,
                  knowns=template_name, sequence=target_name,
                  assess_methods=(assess.DOPE,
                                  #soap_protein_od.Scorer(),
                                  assess.GA341))
    a.starting_model = 1
    a.ending_model = 4
    a.make()


    


# pir_file_path,modified_pdb_file_path,target_name,template_name = get_alignment_pir(pdb_code,pdb_file_path,ligand_name_list,output_path)


# make_mutation

get_modeller(pir_file_path,target_name,template_name,output_path)



    
    
    







# PUT TEAM MEETING SLIDES IN FOLDER ON MICROSOFT TEAMS


        
        
        
#         for i in range(1,len(sequence)):
#                 if template_sequence[i]=='/' and (template_sequence[i-1] in ['/','-']):
#                     template_sequence[i] = '-'
#             template_sequence = ''.join(template_sequence)
            

    
        
            # REMOVE / ON LAST CHAIN
            # FIX OUTPUT OF THIS TO BE PROPERLY FORMATTED
            
#             ali_text.append
#             ali_text.append(f'>P1;{record.id}\n')
#             ali_text.append(f'
#             ali_text.append(template_sequence + '\n\n')
    
#     n = 75
#     ali_text = [ali_text[i:i+n] for i in range(0, len(ali_text),n)]
#     ali_text.insert(0,f'>P1;ABCD')
#     ali_text.insert(1,f'asdfadsfadsfasdfasdfadsf')
#     print(ali_text)
#     print(type(ali_text))
#     ali_text = '\n'.join(ali_text)        
                            
    




# hetatm_name_list = ['IRE-A']

# output_path = "/Users/rcheeter/Desktop/PGN_MODELLER_OUTPUT"
# pdb_file_path = f"{output_path}/4wkq.pdb"
# pir_file_path = f"{output_path}/zsample.ali"

# if not os.path.exists(output_path):
#     os.makedirs(output_path)
#     print(f'● NOTE: creating output path directory: [{output_path}]')
            
# with open(pdb_file_path,'r') as pdb_file,open(pir_file_path,'w') as pir_file:
#         sequences = SeqIO.parse(pdb_file,'pdb-atom')
#         SeqIO.write(sequences,pir_file,'pir')
    
        

# ali_text = []
# with open(pdb_file_path, 'r') as pdb_file:
#     for record in SeqIO.parse(pdb_file, 'pdb-atom'):
#         ali_text.append(f'>P1;{record.id}\n')
#         ali_text.append(str(record.seq))
        
#         print('>' + record.id)
#         print(record.seq)
        
# open(pir_file_path,'w').write(''.join(ali_text)) # write PDB to a new file in output path




rdpdb___459W> Residue type  IRE not recognized. 'AutoModel' model building
              will treat this residue as a rigid body.
              To use real parameters, add the residue type to ${LIB}/restyp.lib,
              its topology to ${LIB}/top_*.lib, and suitable forcefield
              parameters to ${LIB}/par.lib.
fndatmi_285W> Only      296 residues out of      297 contain atoms of type  CA
              (This is usually caused by non-standard residues, such
              as ligands, or by PDB files with missing atoms.)
fndatmi_285W> Only      296 residues out of      297 contain atoms of type  CA
              (This is usually caused by non-standard residues, such
              as ligands, or by PDB files with missing atoms.)

check_ali___> Checking the sequence-structure alignment. 

Implied intrachain target CA(i)-CA(i+1) distances longer than  8.0 angstroms:

ALN_POS  TMPL  RID1  RID2  NAM1  NAM2     DIST
----------------------------------------------
END OF TABLE
read_

In [ ]:
import re
import numpy as np
string = 'AMGEAPNQALLRILKETEFKKIKVLGSXXXGTVYKGLWIPEGEKVKIPVAIKEXXXXXSPK'
print(str.find('X'))
print(str[27])

# missing_residue_indices = [i.start() for i in re.finditer('X', string)]
# string = list(string)
# string[missing_residue_indices] = '/'

# string = ''.join(string)

sequence = list(sequence.replace('X','/'))
for i in range(1,len(string)):
    if sequence[i]=='/' and (sequence[i-1] in ['/','-']):
        sequence[i] = '-'
sequence = ''.join(sequence)

# y = re.finditer('X',string)
print(string)